In [1]:
import pandas as pd
import joblib



In [2]:
df = pd.read_csv("../data/ev_charging_data_preprocessed.csv")
model = joblib.load("../models/ev_demand_model.pkl")


In [3]:
features = ["hour", "station_id", "day_of_week", "is_weekend", "month"]
X = df[features]


In [4]:
df["predicted_demand"] = model.predict(X)
df.head()


,date,hour,station_id,energy_kwh,day_of_week,is_weekend,month,predicted_demand
0,2025-01-01,0,0,10.63,2,0,1,14.255728
1,2025-01-01,0,1,17.10,2,0,1,14.227801
2,2025-01-01,0,2,18.01,2,0,1,14.199873
3,2025-01-01,0,3,10.33,2,0,1,14.171945
4,2025-01-01,0,4,14.38,2,0,1,14.144018


In [5]:
STATION_CAPACITY = 40  # kWh per hour


In [6]:
df["peak_load"] = df["predicted_demand"].apply(
    lambda x: 1 if x > STATION_CAPACITY else 0
)


In [7]:
peak_summary = df[df["peak_load"] == 1].groupby("hour").size()
peak_summary


Series([], dtype: int64)

In [8]:
def recommendation(row):
    if row["predicted_demand"] > STATION_CAPACITY:
        return "Shift charging to off-peak hours"
    else:
        return "Normal operation"

df["recommendation"] = df.apply(recommendation, axis=1)
df[["hour", "predicted_demand", "recommendation"]].head()


,hour,predicted_demand,recommendation
0,0,14.255728,Normal operation
1,0,14.227801,Normal operation
2,0,14.199873,Normal operation
3,0,14.171945,Normal operation
4,0,14.144018,Normal operation
